In [ ]:
%matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pickle
import cv2
np.set_printoptions(formatter={'float': '{: 0.7f}'.format})

In [ ]:
data=[]
#name='../../data/trace_data_fw_and_bw.pkl'
#name='../../data/trace_data_yaw_turn.pkl'
name='../../data/trace_data_u_shape.pkl'
#name='../../data/trace_data_only_pitch.pkl'
with open(name,'rb') as fd:   
    while 1:
        try:
            data.append(pickle.load(fd))
        except EOFError:
            break

In [ ]:
data[0]

In [ ]:
def roty(a):
    R_y = \
        np.array([\
            [np.cos(a),    0,      np.sin(a)  ],
            [0,                     1,      0         ],
            [-np.sin(a),   0,      np.cos(a)  ]
                    ])
    return R_y
def rotx(a):
        ca = np.cos(a)
        sa = np.sin(a)

        R_x = \
            np.array([  [   1,  0,  0   ],
                        [   0,  ca, -sa ],
                        [   0,  sa, ca  ],
                        ])
        return R_x

def rotz(a):
    ch = np.cos(a)
    sh = np.sin(a)
    Rz = np.array([
        [   ch,     -sh,    0],
        [   sh,     ch,     0],
        [   0,      0,      1]])
    return Rz


def get_rot(yaw,pitch,roll):
    return rotz(np.radians(yaw)) @ roty(np.radians(pitch)) @ rotx(np.radians(roll))

print(get_rot(0,45,0))

In [ ]:
__TR = np.array([\
        [   1.0,  0,  0,  0,  ],
        [   0,  1,  0,  0,  ],
        [   0,  0,  1,  0,  ]])


In [ ]:

def triangulate(M,bl,xl,yl,xr,yr):
    p1 = np.array([xl,yl],dtype=float)
    p2 = np.array([xr,yr],dtype=float)
    T=np.array([[bl,0,0]]).T 
    prjl=M @ __TR
    prjr=__TR.copy()
    prjr[0,3]=bl
    prjr=M @ prjr
    p=cv2.triangulatePoints(prjl,prjr,p1,p2)
    p=p/p[3,0]
    x,y,z = p.flatten()[:3]
    ### convert from opencv to our coordinates which is z-up x-forward y-left
    #   opencv coordinate system is described here:  
    #   http://homepages.inf.ed.ac.uk/rbf/CVonline/LOCAL_COPIES/OWENS/LECT9/node2.html  
    return np.array([[-z,-y,x]]).T


In [ ]:
BL=0.122
W,H=1280,1024
ypr=(0,90,0)
f=W/2
sz=(W,H)
M = np.array([\
        [   f, 0,  sz[0]/2   ],
        [   0,  f, sz[1]/2   ],
        [   0,  0,  1,  ]])

P=triangulate(M,BL,W/2+0,H/2+0,W/2+0.2,H/2+0)
p1=get_rot(*ypr) @ P

In [ ]:
p1

In [ ]:
f=W/2
sz=(W,H)
M = np.array([\
            [   f, 0,  sz[0]/2   ],
            [   0,  f, sz[1]/2   ],
            [   0,  0,  1,  ]])


In [ ]:
zrange=p1[2,0]
pix1=np.array([[W/2,H/2,1]]).T
loc1=np.linalg.inv(M) @ pix1 * zrange
loc1[2,0]

In [ ]:
new_ypr = (90,0,0)
ref_ypr = (0,0,0)
R=get_rot(*new_ypr) @ get_rot(*ref_ypr)

In [ ]:
pix2=np.array([[W/2+20,H/2,1]]).T
T=np.linalg.inv(M) @ pix2 * zrange - R@loc1
DC=-R@T #delta camera
DC

In [ ]:
#opencv to water
RO=get_rot(-90,0,-90)
class Tracer(object):
    def __init__(self, M):
        self.current_loc=np.array([0,0.])
        self.last_rel_loc=np.array([0,0.])
        self.M=M
        self.ref_pix=None
    def feed(self,zrange,new_ref,ypr,x,y):
        if new_ref or self.ref_pix is None:
            self.ref_ypr=ypr
            self.current_loc+=self.last_rel_loc
            self.ref_pix=np.array([[x,y,1.0]]).T
        MI=np.linalg.inv(self.M)
        loc1=get_rot(*self.ref_ypr).T @ RO @ MI @ self.ref_pix * zrange
        pix2=np.array([[x,y,1.0]]).T
        loc2=get_rot(*ypr).T @ RO @ MI @ pix2 * zrange
        #print(loc2)
        #T=np.linalg.inv(M) @ pix2 * zrange - get_rot(*ypr)@loc1
        #DC=(-R@T).flatten()
        DC=(loc2-loc1).flatten()
        self.last_rel_loc=DC[:2]
        return self.current_loc+DC[:2]


In [ ]:
tr=Tracer(M)
for i in range(3):
    print(i,tr.feed(10.0,False,(0,-45.0+0,0),W/2.-i*0,H/2.0-i*1))

In [ ]:
class StereoTracer(object):
    def __init__(self,M,BL,start_ypr):
        self.tracer=Tracer(M,start_ypr)
        self.BL=BL
        self.M=M
    def feed(self,new_ref,ypr,xl,yl,xr,yr):
        P=triangulate(self.M,self.BL,xl,yl,xr,yr)
        P1=get_rot(*ypr) @ P
        zrange=P1[2,0]
        #print('---',zrange)
        if xr-xl<0.1:
            #print('not good for stereo',xr-xl)
            return None
        return self.tracer.feed(zrange,new_ref,ypr,xl,yl)

In [ ]:
get_ypr = lambda i:(data[i]['yaw'],data[i]['pitch'],data[i]['roll'])
data[0]

In [ ]:
data[173]

In [ ]:
tr=Tracer(M)
last_ref_cnt=-1
ps=[]
xs=[]
ys=[]
locs=[]
for i in range(0,len(data)):
    d=data[i]
    if 'sonar' in d:
        new_ref=d['ref_cnt']!=last_ref_cnt
        last_ref_cnt=d['ref_cnt']
        pl=d['pt_l']
        xs+=[pl[0]]
        ys+=[pl[1]]
        ps+=[d['pitch']]
        ret=tr.feed(d['sonar'][0]/100,new_ref,get_ypr(i),pl[0],pl[1])
        locs.append(ret)
        #print(i,d['sonar'][0],d['pitch'],)
locs=np.array(locs)

In [ ]:
plt.figure()
#plt.plot(ps)
#plt.plot(xs)
#plt.plot()
#plt.plot(locs[:,1])
plt.plot(locs[:,0],locs[:,1])
plt.axis('equal')
#plt.legend(['ps','xs','ys'])

In [ ]:
st=StereoTracer(M,BL,get_ypr(100))
last_ref_cnt=-1
for i in range(100,120):
    d=data[i]
    new_ref=d['ref_cnt']!=last_ref_cnt
    last_ref_cnt=d['ref_cnt']
    pl,pr=d['pt_l'],d['pt_r']
    #print(i,pr[0]-pl[0],st.feed(new_ref,get_ypr(i),pl[0],pl[1],pr[0],pr[1]))